<center>
<h1> Introduction to Numerical Computing with the&nbsp;Perl&nbsp;Data&nbsp;Language (PDL)</h1>
<h3>Zaki Mughal</h3>
<h3>PAUSE: ZMUGHAL ; IRC: sivoais<h3>
<h3>DC.pm</h3>
<h4>The Perl and Raku Conference 2023; Toronto, CA; 2023 July 11th-13th</h4>
</center>

In [ ]:
use strict;
use warnings;
use constant IN_IPERL => !! $ENV{PERL_IPERL_RUNNING};
no if IN_IPERL, warnings => 'redefine'; # fewer messages when re-running cells

use feature qw(say postderef state);
use Syntax::Construct qw( /r );
use Capture::Tiny qw(capture_stdout);
use Path::Tiny;

use PDL;
use PDL::Graphics::Gnuplot;
IPerl->load_plugin('PDLGraphicsGnuplot');

use Data::Printer ( output => 'stderr', return_value => 'void', filters => ['PDL'] );
use Data::Printer::Filter::PDL ();

sub _err { shift =~ s/^\Q.. at reply\E .*//xsmr }
sub _pdldoc {
    my @doc = @_;
    my $output = capture_stdout { system('pdldoc', @doc) };
    $output =~ s/\s*Docs\s*from.*//sr;
}

sub _info {
    join "\n", map { "@{[$_->info]}\n@{[ $_->string =~ s/^\n*//mgr ]}" } @_;
}

();

<center><h1>What is PDL</h1><center>

- A DSL and library for working with multidimensional numerical data.

- Container data type for homogenous n-dimensional array:
  * Packed binary data,
  * With a list of dimensions,
  * Of a single type;
  * Called an **"ndarray"** (older docs use "piddle").

- Dimensions- and type-aware functions.
  * Code-generation DSL called PP (generates C + XS).
  * Signatures with dimensions.
  * Dimensions are used to perform **"broadcasting"** (older docs use "threading")

# Installing PDL

- Several dependencies.
- If you can, install via [package manager](https://repology.org/project/perl:pdl/versions).
  * Debian package is reasonably up to date.

- Otherwise, you need:
  * Required: C compiler,
  * Optional: Fortran compiler,
  * Optional (compute): `libgsl`, `libproj` development headers,
  * Optional (I/O): `libgd`, `netpbm`, `libhdf4`, `libcfitsio` development headers.
* Tip: `MAKEFLAGS=-j$(nproc) cpanm PDL`

<center><h1>Let's jump right in!</h1><center>

In [ ]:
use PDL;

my $p = pdl(1);
my $q = pdl(2);
my $r = $p + $q;
say $r; ();

In [ ]:
say 'Number of dims: ', $p->ndims;

# as Perl list
say 'Dims list:      ', join(' ', '(', $p->dims, ')');

# as another ndarray
say 'Shape:          ', $p->shape;
();

In [ ]:
say $p->info; ();

In [ ]:
say $p->type;
say PDL::Core::howbig($p->type); ();

<center><h1>More data</h1></center>

In [ ]:
$p = pdl(1,2);
$q = pdl(3,4);

say $p->info; say $p;
say $q->info; say $q;

$p + $q;

In [ ]:
$p = pdl(1,2);
$q = pdl(3,4,5);

say _info($p, $q); ();

In [ ]:
# YOW!
$r = eval { $p + $q; }; say _err($@); ();

In [ ]:
# $ pdldoc plus
_pdldoc('plus');

In [ ]:
_pdldoc(qw(-s plus));

- `a()`: input, empty dims,
- `b()`: input, empty dims,
- `[o]c()`: output, empty dims.

Thus the `plus` fuction signature indicates it works *element-wise*.

In [ ]:
say _pdldoc('PDL::Ops'); ();

In [ ]:
### Match dimensions
$p = pdl(1,2);
$q = pdl([ [3],[4],[5] ]);
# $q = transpose( pdl([  3,  4,  5  ]) );

say _info( $p, $q  ); ();
say '$q(0,1): ', $q->at(0,1), "\n"; # extract single element as Perl scalar by indices

say _info(

    $p + $q

); ();

In [ ]:
$p = pdl(1,2);
$q = pdl(   3 , 4 , 5 )->dummy(0); # add dummy of size 1 at position 0

say _info( $p, $q  );

$r = $p + $q;

In [ ]:
say pdl(1..3)->dummy(1024)->ndims; (); # add dummies of size 1 at position 1-1024

In [ ]:
my $even_mask = $r % 2 == 0;

say $even_mask;

say 'any: ', any( $even_mask );
say 'all: ', all( $even_mask ); ();

In [ ]:
which( $even_mask ) if any( $even_mask );

In [ ]:
sequence($r); #->flat;

In [ ]:
$r->where( $r % 2 == 0 );

In [ ]:
$r->where( $r % 2 == 0 ) .= 42;
$r;

In [ ]:
$r->where( $r % 2 == 3 );

In [ ]:
$r = zeroes(3,3,3);
# ones($r);
# sequence($r);
# xvals($r);
# yvals($r);
# zvals($r);
# rvals($r);
# random($r);

In [ ]:
my $p = zeroes(9,9);
# my $x = $p->xlinvals(0, 1);
# my $y = $p->ylinvals(-0.5, 0.5);

# my $z = $x**2 + $x * $y;

In [ ]:
identity(3);
# stretcher(pdl( [3,2,1] ) );

<center><h1>Slicing and Dicing</h1></center>

In [ ]:
use Data::TestImage;
use PDL::IO::Pic;
my $mandrill = rim( Data::TestImage->get_image('mandrill') . '');

say _info($mandrill); ();

In [ ]:
say _pdldoc('rim'); ();

In [ ]:
use PDL::Graphics::Gnuplot;

my $gp = gpwin();
$gp->image( $mandrill, { square => 1 } );
$gp;

In [ ]:
$gp->image( $mandrill->slice('0:255,0:255'), { square => 1 } );
$gp;

In [ ]:
my $mandrill_TL_gfx = $mandrill->slice('0:255,-256:-1');
say $mandrill_TL_gfx->info;

$gp->image( $mandrill_TL_gfx, { square => 1 } );
$gp;

In [ ]:
my $mandrill_rgb_split = $mandrill * identity(3,3)->slice('*,*');
say $mandrill_rgb_split->info;

say $mandrill_rgb_split->clump(0..1)->sumover; ();

In [ ]:
$Devel::IPerl::Plugin::PDLGraphicsGnuplot::IPerl_compat = 0;

In [ ]:
$gp = gpwin( "png", output => 'mandrill-mp.png' );
$gp->multiplot( layout => [3,1] );
$gp->plot( with => 'image', $mandrill_rgb_split->slice(',,,(0)'), { square => 1 });
$gp->plot( with => 'image', $mandrill_rgb_split->slice(',,,(1)'), { square => 1 });
$gp->plot( with => 'image', $mandrill_rgb_split->slice(',,,(2)'), { square => 1 });
$gp->end_multi;
$gp->close;

IPerl->png( path('mandrill-mp.png')->slurp_raw );

In [ ]:
$gp = gpwin( "png", output => 'mandrill-mp-dog.png' );
$gp->multiplot( layout => [3,1] );
$gp->plot( with => 'image', $_, { square => 1 }) for dog($mandrill_rgb_split);
$gp->end_multi;
$gp->close;

IPerl->png( path('mandrill-mp-dog.png')->slurp_raw );

In [ ]:
$Devel::IPerl::Plugin::PDLGraphicsGnuplot::IPerl_compat = 1;

In [ ]:
$gp = gpwin();
my $mandrill_mv_dims = $mandrill_rgb_split->mv(-1,1);
say $mandrill_mv_dims->info;

# my $mandrill_panel = $mandrill_mv_dims->reshape(512*3,512,3);
my $mandrill_panel = $mandrill_mv_dims->clump(2);

say $mandrill_panel->info;

$gp->image( $mandrill_panel, { square => 1 } );

$gp;

In [ ]:
$gp = gpwin();

my $mandrill_panel = $mandrill_rgb_split->mv(-1,2)->clump(1..2);
# ->mv(-1,1)

say $mandrill_panel->info;

$gp->image( $mandrill_panel, { square => 1 } );

# say $mandrill_panel->slice(':,0:4,:')->avgover;

$gp;

In [ ]:
$gp = gpwin();
my @range = (0,255); my $steps = 16;
my ($hist_x, $hist_count) = hist( $mandrill->slice(',,0'), @range, $steps );

$gp->plot( with => 'steps', $hist_x, $hist_count, { xrange => \@range } );

say cat($hist_x, $hist_count);

$gp;

In [ ]:
my $threshold = 225;

$gp->options( arrow1 => [
    at => "$threshold, graph 0",
    to => "$threshold, graph 1",
    'nohead lc "red" dt 2' ]);

$gp->replot;

$gp;

In [ ]:
my $mandrill_thresh = $mandrill->copy;
$mandrill_thresh->where( $mandrill_thresh->slice(',,0') <= $threshold ) .= 0;

$gp = gpwin();
$gp->image(
    $mandrill->glue(0,
        zeroes(10),
        $mandrill_thresh),
    { justify => 1 } );
$gp;

# Special functions

In [ ]:
use PDL::Constants qw(PI);
use PDL::GSL::CDF;
use PDL::GSL::INTEG;

In [ ]:
my $xvals = zeroes(256)->xlinvals(-4,4);
my $sd    = 1+sequence(3)->dummy(0);
$gp = gpwin( enhanced => 1 );
$gp->option( key => 'reverse Left left font ",16"' );
$gp->plot(
    legend => [ map { "{/Symbol s} = $_" } $sd->flat->unpdl->@* ],
    $xvals, gsl_cdf_gaussian_P( $xvals, $sd ) );
$gp;

In [ ]:
sub gauss_pdf {
  my ($x) = @_;
  state $constant = 1 / sqrt(2*PI);
  return $constant * exp(- $x**2 / 2);
}

my ($cdf_integ) = gslinteg_qagil( \&gauss_pdf, $xvals , 1e-16, 0, 1000);
my $cdf_direct = gsl_cdf_gaussian_P( $xvals, 1 );
$gp = gpwin( enhanced => 1 );
$gp->option( key => 'reverse Left left font ",16"' );
$gp->plot(
    legend => "{/Symbol s} = 1 (direct)",
        $xvals, $cdf_direct,
    legend => "{/Symbol s} = 1 (integration)",
        $xvals, $cdf_integ + 0.1,
);
IPerl->display($gp);

say join " ", minmax(abs($cdf_integ - $cdf_direct));

# PDL::Stats

In [ ]:
use Data::Frame::Examples qw(iris);

my $iris_data = iris;

In [ ]:
my $iris_labels = $iris_data->column('Species')->{PDL}; # using non-public API here, do not use!

my $iris_numeric = cat(map $iris_data->nth_column($_), 0..3);
$iris_numeric->info;

In [ ]:
use PDL::Stats::GLM;

my %pca = pca($iris_numeric);
p %pca;

# Alternative: LAPACK SVD from PDL::LinearAlgebra

In [ ]:
$gp = gpwin();
my $pc_idx = sequence($pca{pct_var}->dim(0));
$gp->plot(
    title => 'Percent variance for PC n',
    boxwidth => 0.5, xtics => { out => 1, locations => [$pc_idx->min,1,$pc_idx->max] },
    with => 'boxes', fs => 'solid', $pc_idx, $pca{pct_var} );
$gp;

In [ ]:
my $comps = [0,1];
my $z = $iris_numeric->stddz;

# transformed normed values
my $scores = sumover($pca{eigenvector}->slice([], $comps) * $z->transpose->dummy(1))->transpose;
$z = $z->slice([], $comps)->sever;
$gp = gpwin(); $gp->option( justify => 1, key => 'reverse Left left font ",16" outside' );
$gp->plot( xlabel => 'PC1', ylabel => 'PC2',
        map {
            my $label = $_;
            my $subset = $scores->slice(which($iris_labels == $label), []);
            ( with => 'points',
                legend => [$iris_data->column('Species')->levels->[$label]],
                $subset->slice(',0'), $subset->slice(',1') )

        } $iris_labels->uniq->list
);
$gp;

In [87]:
=pod

use PDL::Demos::Sound;
use List::Util qw(mesh);
my @row_tones = (697, 770, 852, 941);
my @col_tones = (1209, 1336, 1477);

my @keypad = ( qw(
    1 2     3
    4 5     6
    7 8     9
    * 0 ), '#' );
my %keypad = mesh(\@keypad, [0..$#keypad]);

open (my $pipe, '|-', $PDL::Demos::Sound::player, @$PDL::Demos::Sound::player_args)
    or die "Can not start a sound player.  Demo failed.\n";
binmode $pipe;
my $n         = 4000;
my $samples   = sequence $n;
my $raw_sound = byte(zeroes($n));
my $amplitude = 80;
for my $key ('1', '2', '3', '4') {
    my $key_idx = $keypad{$key};
    my $r = $row_tones[ $key_idx / 3 ];
    my $c = $col_tones[ $key_idx % 3 ];
    $raw_sound = byte($amplitude*(1
        + sin($samples*$r*2*PI/$n)
        + sin($samples*$c*2*PI/$n)
    ));
    print $pipe ${$raw_sound->get_dataref};
}
close $pipe;

=cut

In [ ]:
system('echo "demo opencv" | perldl 2>/dev/null >&2'); # from PDL::OpenCV

# More features

- <https://metacpan.org/pod/PDL::ParallelCPU>
- <https://metacpan.org/pod/PDL::Dataflow>
- <https://metacpan.org/pod/PDL::BadValues>
- <https://metacpan.org/pod/PDL::Demos>

# More talks/videos

- [David Mertens - Introduction to the Perl Data Language](https://www.youtube.com/watch?v=rf1yfZ2yUFo)
- [Maggie Xiong - Statistics and data mining with Perl Data Language](https://www.youtube.com/watch?v=DFX_cNB97yQ)
- [David Mertens - Interactive Data Analysis with Prima and PDL](https://www.youtube.com/watch?v=IE-vnnRWiOg)
- [PDL::Graphics::Gnuplot Tutorial](https://www.youtube.com/watch?v=hUXDQL3rZ_0)
- [Carey Witkov - Higgs meets Perl](https://www.youtube.com/watch?v=I7OMJKguseo)

And for another second dose of this kind of thing, head over to [Introduction to TensorFlow in Perl](https://tprc2023.sched.com/event/1LhoZ/introduction-to-tensorflow-in-perl) at this conference.

# Community

- PDL mailing list (<https://sourceforge.net/p/pdl/mailman/>) and IRC (`#pdl` on MagNET irc.perl.org)
- Downstream PDL libraries: <https://github.com/PDLPorters/devops/blob/master/data/project.yml>
- Use it, submit bugs and feature requests.